## 1. Code base

In [107]:
import importlib
import torch
from torch_geometric.data import DataLoader
import networkx as nx
from torch_geometric.data import Data
import time
import pickle
from torch_geometric.data import InMemoryDataset
import numpy as np
import traceback

      
    
def writeAdjacencyMatrixToDisk(G, filename='temp_adjacency_matrix.txt'):
    """
        Transform to networkx dataset

        possible formats: GML, Adjacency matrix, ..
        start by Adjcency list 
             --> (ignoring edge/node features)
             --> line format: source target target2 target3 ... 
        later we can improve this...
    """
    f = open(filename,'w')
    _ni=-1
    newline = False
    theline = []
    careturn = ""
    for ei in range(G.edge_index.size()[1]):
        if int(G.edge_index[0,ei].item()) != _ni:
            newline=True
            _ni=int(G.edge_index[0,ei].item())
            
        else:
            newline=False
            
            
        ni = str(G.edge_index[0,ei].item())
        vi = str(G.edge_index[1,ei].item())
        if newline:
            f.write(''.join(theline))
            #print(''.join(theline))
            #print(" --> "+str(_ni))
            theline =[]
            theline.append(careturn+ni+" ")
            theline.append(vi+" ")
            careturn = "\n"
        else:
            theline.append(vi+" ")
        # print("({},{})".format(ni,vi))
    
    
def nx_createNxGraphInMem(G):
    """
        Transform to networkx dataset

        possible formats: GML, Adjacency matrix, ..
        start by Adjcency list 
             --> (ignoring edge/node features)
             --> line format: source target target2 target3 ... 
        later we can improve this...
    """
    g = nx.MultiGraph()
   
    for ei in range(G.edge_index.size()[1]):    
        ni = str(G.edge_index[0,ei].item())
        vi = str(G.edge_index[1,ei].item())
        g.add_edge(ni,vi)
    return g
    
def nx_verifyEdges(G, g):
    for ei in range(G.edge_index.size()[1]):
        ni = str(G.edge_index[0,ei].item())
        vi = str(G.edge_index[1,ei].item())
        if (ni,vi,0) not in list(g.edges):
            if (vi,ni,1) not in list(g.edges):
                print("Error {} not in networkx graph".format((ni,vi)))
            
        

def nx_compute_edge_betweenness(G):
    
    #print(list(G.edges)[:10])
    G_components = nx.connected_component_subgraphs(G)
    G_mc = list(G_components)[0]  
    eb_dict_res = {}
    eb_dict = nx.edge_betweenness_centrality(G_mc)
    
    # if there are more connected components...
    if len(list(G_components))>1:
        print("WARNING connected components: ",len(list(G_components)))
    
    eb_dict_res.update(eb_dict)
    
        
    return eb_dict_res

def nx_compute_node_betweenness(G):
    
    #print(list(G.edges)[:10])
    G_components = nx.connected_component_subgraphs(G)
    G_mc = list(G_components)[0]  
    eb_dict_res = {}
    eb_dict = nx.betweenness_centrality(G_mc)
    
    # if there are more connected components...
    if len(list(G_components))>1:
        print("WARNING connected components: ",len(list(G_components)))
    
    eb_dict_res.update(eb_dict)
    
        
    return eb_dict_res


def update_edge_betweenness(G, eb_dict):
    """
        FOR UNDIRECTED GRAPHS
    
        G.edge_attr must contain the edge betweenness values 
        for each edge
        
        G.y must contain it also.. (it is a copy of the edge betweenness..)
        this could help the training phase
        
        Size restrictions:
        - Given the size of the graphs, is it better to just transform the 
        object instead to write a new one?
        - also just use G.y? but for GNN algorithms..not sure
        
        new_edg_attr will be size [num edges, 1]
        and must be sorted in accordance to G.edge_index
        
    
    """
    
    new_edg_attr = []
    for i in range(len(G.edge_index[0])):
        ni = G.edge_index[0][i]
        vi = G.edge_index[1][i]
        
        if ni and vi:
            ni=str(ni.item())
            vi=str(vi.item())
            #print((ni,vi))
            try:
                new_edg_attr.append([eb_dict[(ni,vi)]])
            except:
                try:
                    new_edg_attr.append([eb_dict[(vi,ni)]])
                except:
                    #print("ERROR {} and {} not found!".format((ni,vi),(vi,ni)))
                    new_edg_attr.append([0])
        else:
            new_edg_attr.append([0])

    new_edg_attr = torch.FloatTensor(new_edg_attr)
    
    #newG = Data(
    #    x=G.x, 
    #    edge_index=G.edge_index, 
    #    edge_attr=new_edg_attr,
    #    y=new_edg_attr)
    
    #G.edge_attr = new_edg_attr
    G.y = new_edg_attr
    
    return G


def update_node_betweenness(G, eb_dict):
    """
        Get nodes keys from eb_dict and get their betweenness centrality
        G.y will have all centralities of al lnodes following the order
        of a list of the nodes sorted by id

        add spaces in between!

    """
    betweennesses = []
    nodes = sorted([int(k) for k in eb_dict.keys()])
    for node in range(nodes[-1]+1):
        try:
            betweennesses.append(eb_dict[str(node+1)])    
        except:
            betweennesses.append(0.0)
            
    G.y = torch.FloatTensor(betweennesses)
    return G

def get_betweenness_into_dict(G):
    """
        FOR UNDIRECTED GRAPHS
    """
    
    eb_dict ={}
    for i in range(len(G.edge_index[0])):
        ni = G.edge_index[0][i]
        vi = G.edge_index[1][i]
        
        if ni and vi:
            ni=str(ni.item())
            vi=str(vi.item())
            eb_dict[(ni,vi)] = float(G.y[i].item())
    return eb_dict


def pyTorchGeometricDatasetToNx(G,suffix=0):
    """
        Alternatives:
            - to disk, to nx, then dict of betweenness
            - transform in memory
            - directly pickle a G object with the betweenness
    """
    prefix = 'temp_aj_m'
    # 1. PyTorch Geometric graph -> nx -> compute betweenness 
    #             -> PyTorch Geom with target the betweenness-------
    # Transform to networkx graph
    # write to adjacency matrix on disk
    writeAdjacencyMatrixToDisk(G, filename=prefix+str(suffix)+'.txt')

    # load into a networkx graph object
    g2 = nx.read_adjlist(prefix+str(suffix)+'.txt')
    #g2 = nx_createNxGraphInMem(G)
    
    return g2

def computeBetweenness(G,suffix=0):
    """
        Alternatives:
            - to disk, to nx, then dict of betweenness
            - transform in memory
            - directly pickle a G object with the betweenness
    """
    prefix = 'temp_aj_m'
    # 1. PyTorch Geometric graph -> nx -> compute betweenness 
    #             -> PyTorch Geom with target the betweenness-------
    # Transform to networkx graph
    # write to adjacency matrix on disk
    writeAdjacencyMatrixToDisk(G, filename=prefix+str(suffix)+'.txt')

    # load into a networkx graph object
    g2 = nx.read_adjlist(prefix+str(suffix)+'.txt')
    #g2 = nx_createNxGraphInMem(G)

    # compute node betweenness centrality
    eb_dict = nx_compute_node_betweenness(g2)
    #print("eb_dict",eb_dict)
    
    # write node betweenness back to PyTorch Geometric graph
    update_node_betweenness(G,eb_dict)
    #return G
    


class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, name, transform=None, pre_transform=None):
        f = open(name, 'rb')
        self.data = pickle.load(f) 
        #data_list = [G.x, G.edge_index, G.test_mask, G.train_mask, G.val_mask, G.y, G.batch]
        f.close()
        #print("root ", root, " name ", name)
        #print("setting a self.name in the object!")
        #self.name = name
        #print(dir(self))
        #super(MyOwnDataset, self).__init__(root,transform, pre_transform)
        #self.data = torch.load(self.processed_paths[0])
        
        
    
    
    @property
    def raw_file_names(self):
        return ['PPI0.pickle']

    @property
    def processed_file_names(self):
        return ['PPI0']

    def download(self):
        # Download to `self.raw_dir`.
        return True

    def process(self):
        # Read data into huge `Data` list.
        
        # unpickle the graph
        print("going to unpickle")
        f = open(self.name, 'rb')
        G = pickle.load(f) 
        #data_list = [G.x, G.edge_index, G.test_mask, G.train_mask, G.val_mask, G.y, G.batch]
        f.close()
        
        #if self.pre_filter is not None:
        #    data_list [data for data in data_list if self.pre_filter(data)]

        #if self.pre_transform is not None:
        #    data_list = [self.pre_transform(data) for data in data_list]

        #data, slices = self.collate(data_list)
        torch.save(G, self.processed_paths[0])
        
        
class MyOwnDataset2():
    def __init__(self,  name, transform=None, pre_transform=None):
        f = open(name, 'rb')
        self.data = pickle.load(f) 
        f.close()
    
def loadDataset(collection, name=None, split=None):
    # import datasets
    themodule = importlib.import_module("torch_geometric.datasets")
    # get the function corresponding to collection
    method_to_call = getattr(themodule, collection)
    try:
        if name:
            return method_to_call(root='./data/'+str(collection), name=name)
        elif split:
            return method_to_call(root='./data/'+str(collection), split=split)
        else:
            return method_to_call(root='./data/'+str(collection)) 
    except Exception e:
        traceback.print_tb()
        if name:
            return method_to_call( name=name)
        elif split:
            return method_to_call(split=split)
        else:
            return method_to_call()         
        
    
def createDataset(x, edge_index):
    return Data(x=x, edge_index=edge_index)
    

def createDatasetFromNX(g, xlen, undirected=True):
    # get edge list
    edges = g.edges
    edge_list_1 = []
    edge_list_2 = []
    for e in edges:
        # node id must be an int
        edge_list_1.append(int(e[0])) 
        edge_list_2.append(int(e[1]))
        if undirected:
            edge_list_1.append(int(e[1])) 
            edge_list_2.append(int(e[0]))
            
        
    edge_index = torch.tensor([ edge_list_1,
                                edge_list_2], dtype=torch.long)
    
    # create single 1 feature for each node
    n = xlen
    x = [[1.0] for i in range(n)]
    x = torch.tensor(x, dtype=torch.float)
    
    return createDataset(x, edge_index)
                         
def createDatasetFromNXwithTarget(g,y,xlen, undirected=True):
    dataset =  createDatasetFromNX(g,xlen, undirected)
    y = torch.FloatTensor(y)
    dataset.y = y 
    return dataset
                   

SyntaxError: invalid syntax (<ipython-input-107-cebeee1e7600>, line 322)

## 2. Random Graph datasets

### Tests

In [2]:
# generate random graphs
er = nx.erdos_renyi_graph(100, 0.15)
ws = nx.watts_strogatz_graph(30, 3, 0.1)
ba = nx.barabasi_albert_graph(100, 5)
red = nx.random_lobster(100, 0.9, 0.9)
g=er


# compute its node and edge betweenness
nx_betweenness = nx.betweenness_centrality(g)
nx_edge_betweenness = nx.edge_betweenness_centrality(g)
y = [ v for k,v in nx_betweenness.items()]

# verify order of betweenness is the same as  order of edge_list
#print(nx_betweenness)
#print(y)
#-> ok!

# translate into a PyTorch Geometric dataset 
dataset = createDatasetFromNXwithTarget(g,y)

# check dimensions
print(dataset) # check edge_index[1] and y have same length


# save as a pickled object
dname = "er_100_0_15"
i=0
with open(dname+"_"+str(i)+'.pickle','wb') as f:
    pickle.dump(dataset,f)



Data(edge_index=[2, 740], x=[100, 1], y=[100])


### Implementation

In [105]:

def createRandomGraphDataset(g,dname,betweenness, undirected=True):
    
    # compute its node and edge betweenness
    y =[]
    x=[]
    xlen=len(g.nodes())
    
    if betweenness == 'node':
        nx_betweenness = nx.betweenness_centrality(g)
        y = [ v for k,v in nx_betweenness.items()]
    else:
        nx_edge_betweenness = nx.edge_betweenness_centrality(g)
        #y = [ v for k,v in nx_edge_betweenness.items()]
        # we need to double it? because undirected graph show 2 edge each in one direction?
        y = []
        for k,v in nx_edge_betweenness.items():
            y.append(v)
            if undirected:
                y.append(v)
        xlen=len(y)
        
        
    # verify order of betweenness is the same as  order of edge_list
    #print(nx_betweenness)
    #print(y)
    #-> ok!

    # translate into a PyTorch Geometric dataset 
    dataset = createDatasetFromNXwithTarget(g,y,xlen,undirected)
    print(dataset.num_features)

    # check dimensions
    print(dataset) # check edge_index[1] and y have same length
    print(" directed graph: ",dataset.is_directed())
    print("isolated nodes: ",dataset.contains_isolated_nodes())
    print("self loops: ",dataset.contains_self_loops())
    
    # save as a pickled object
    with open(dname+'.pickle','wb') as f:
        pickle.dump(dataset,f)

In [14]:
er = nx.erdos_renyi_graph(100, 0.15)
ws = nx.watts_strogatz_graph(30, 3, 0.1)
ba = nx.barabasi_albert_graph(100, 5)
red = nx.random_lobster(100, 0.9, 0.9)

createRandomGraphDataset(er,'er_100_0_15_nb','node', undirected=False)



1
Data(edge_index=[2, 1516], x=[100, 1], y=[100])
 directed graph:  False
isolated nodes:  False
self loops:  False


In [101]:
from pprint import pprint

class MyOwnDataset2():
    def __init__(self,  root, name, transform=None, pre_transform=None):
        f = open(name, 'rb')
        self.data = pickle.load(f) 
        print(self.data)
        print(self.data.edge_index)
        self.num_features = self.data.num_features
        self.num_classes = 1
        self.filename = name
        # prepare edge_neighbors
        edges_dict = {}
        i=0
        for edge in self.data.edge_index[0]:
            edges_dict[0]= (self.data.edge_index[0][i],
                            self.data.edge_index[1][i])
            i+=1                
        edge_neighbors= [[],[]]
        for edge in edges_dict.keys():
            for node in edges_dict[edge]:
                for edge2 in edges_dict.keys():
                    if edge2 != edge and ( edges_dict[edge2][0] == node or edges_dict[edge2][1] == node ):
                        edge_neighbors[0].append(edge)
                        edge_neighbors[1].append(edge2)
        
        self.data.edge_neighbors = torch.LongTensor(edge_neighbors)
        print()
        print("edge_neighbors")
        pprint(self.data.edge_neighbors)
        f.close()
        
er = nx.erdos_renyi_graph(5, 0.45)

# set undirected to False but later it will be transformed to undirected
createRandomGraphDataset(
    er,
    'precomputed/er_5_0_45_eb',
    'edge', 
    undirected=False)
dataset = MyOwnDataset2(
    root='', 
    name='precomputed/er_5_0_45_eb.pickle')
print(dataset.data.x)


1
Data(edge_index=[2, 7], x=[7, 1], y=[7])
 directed graph:  True
isolated nodes:  True
self loops:  False
Data(edge_index=[2, 7], x=[7, 1], y=[7])
tensor([[0, 0, 1, 1, 1, 2, 3],
        [1, 2, 2, 3, 4, 4, 4]])

edge_neighbors
tensor([], size=(2, 0), dtype=torch.int64)
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])


In [20]:
er = nx.erdos_renyi_graph(100, 0.15)
createRandomGraphDataset(er,'er_100_0_15_nb','node')
er = nx.erdos_renyi_graph(100, 0.45)
createRandomGraphDataset(er,'er_100_0_45_nb','node')
er = nx.erdos_renyi_graph(1000, 0.15)
createRandomGraphDataset(er,'er_1000_0_15_nb','node')
er = nx.erdos_renyi_graph(1000, 0.45)
createRandomGraphDataset(er,'er_1000_0_45_nb','node')


ws = nx.watts_strogatz_graph(30, 3, 0.1)
createRandomGraphDataset(ws,'ws_30_3_0_1_nb','node')
ws = nx.watts_strogatz_graph(100, 3, 0.1)
createRandomGraphDataset(ws,'ws_100_3_0_1_nb','node')
ws = nx.watts_strogatz_graph(1000, 3, 0.1)
createRandomGraphDataset(ws,'ws_1000_3_0_1_nb','node')


ws = nx.watts_strogatz_graph(1000, 10, 0.1)
createRandomGraphDataset(ws,'ws_1000_10_0_1_nb','node')


ba = nx.barabasi_albert_graph(100, 5)
createRandomGraphDataset(ba,'ba_100_5_nb','node')
ba = nx.barabasi_albert_graph(1000, 5)
createRandomGraphDataset(ba,'ba_1000_5_nb','node')



#er = nx.erdos_renyi_graph(4000, 0.15)
#createRandomGraphDataset(er,'er_4000_0_15_nb','node')
#er = nx.erdos_renyi_graph(4000, 0.35)
#createRandomGraphDataset(er,'er_4000_0_35_nb','node')
#ws = nx.watts_strogatz_graph(4000, 3, 0.1)
#createRandomGraphDataset(ws,'ws_4000_3_0_1_nb','node')
#ws = nx.watts_strogatz_graph(4000, 20, 0.1)
#createRandomGraphDataset(ws,'ws_4000_20_0_1_nb','node')
#ba = nx.barabasi_albert_graph(4000, 5)
#createRandomGraphDataset(ba,'ba_4000_5_nb','node')


1
Data(edge_index=[2, 1542], x=[100, 1], y=[100])
 directed graph:  False
isolated nodes:  False
self loops:  False
1
Data(edge_index=[2, 4432], x=[100, 1], y=[100])
 directed graph:  False
isolated nodes:  False
self loops:  False
1
Data(edge_index=[2, 149596], x=[1000, 1], y=[1000])
 directed graph:  False
isolated nodes:  False
self loops:  False
1
Data(edge_index=[2, 449110], x=[1000, 1], y=[1000])
 directed graph:  False
isolated nodes:  False
self loops:  False
1
Data(edge_index=[2, 60], x=[30, 1], y=[30])
 directed graph:  False
isolated nodes:  False
self loops:  False
1
Data(edge_index=[2, 200], x=[100, 1], y=[100])
 directed graph:  False
isolated nodes:  False
self loops:  False
1
Data(edge_index=[2, 2000], x=[1000, 1], y=[1000])
 directed graph:  False
isolated nodes:  False
self loops:  False
1
Data(edge_index=[2, 10000], x=[1000, 1], y=[1000])
 directed graph:  False
isolated nodes:  False
self loops:  False
1
Data(edge_index=[2, 950], x=[100, 1], y=[100])
 directed graph

KeyboardInterrupt: 

In [102]:
# repeat with edge betweenness
er = nx.erdos_renyi_graph(100, 0.15)
createRandomGraphDataset(er,'er_100_0_15_eb','edge', undirected=False)
er = nx.erdos_renyi_graph(100, 0.45)
createRandomGraphDataset(er,'er_100_0_45_eb','edge', undirected=False)
er = nx.erdos_renyi_graph(1000, 0.15)
createRandomGraphDataset(er,'er_1000_0_15_eb','edge', undirected=False)
er = nx.erdos_renyi_graph(1000, 0.45)
createRandomGraphDataset(er,'er_1000_0_45_eb','edge', undirected=False)


ws = nx.watts_strogatz_graph(30, 3, 0.1)
createRandomGraphDataset(ws,'ws_30_3_0_1_eb','edge', undirected=False)
ws = nx.watts_strogatz_graph(100, 3, 0.1)
createRandomGraphDataset(ws,'ws_100_3_0_1_eb','edge', undirected=False)
ws = nx.watts_strogatz_graph(1000, 3, 0.1)
createRandomGraphDataset(ws,'ws_1000_3_0_1_eb','edge', undirected=False)


ws = nx.watts_strogatz_graph(1000, 10, 0.1)
createRandomGraphDataset(ws,'ws_1000_10_0_1_eb','edge', undirected=False)


ba = nx.barabasi_albert_graph(100, 5)
createRandomGraphDataset(ba,'ba_100_5_eb','edge', undirected=False)
ba = nx.barabasi_albert_graph(1000, 5)
createRandomGraphDataset(ba,'ba_1000_5_eb','edge', undirected=False)


#er = nx.erdos_renyi_graph(4000, 0.15)
#createRandomGraphDataset(er,'er_4000_0_15_eb','edge')
#er = nx.erdos_renyi_graph(4000, 0.35)
#createRandomGraphDataset(er,'er_4000_0_35_eb','edge')
#ws = nx.watts_strogatz_graph(4000, 3, 0.1)
#createRandomGraphDataset(ws,'ws_4000_3_0_1_eb','edge')
#ws = nx.watts_strogatz_graph(4000, 20, 0.1)
#createRandomGraphDataset(ws,'ws_4000_20_0_1_eb','edge')
#ba = nx.barabasi_albert_graph(4000, 5)
#createRandomGraphDataset(ba,'ba_4000_5_eb','edge')

1
Data(edge_index=[2, 748], x=[748, 1], y=[748])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 2245], x=[2245, 1], y=[2245])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 74983], x=[74983, 1], y=[74983])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 224755], x=[224755, 1], y=[224755])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 30], x=[30, 1], y=[30])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 100], x=[100, 1], y=[100])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 1000], x=[1000, 1], y=[1000])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 5000], x=[5000, 1], y=[5000])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 475], x=[475, 1], y=[475])
 directed graph:  True
iso

## 3. Benchmark datasets

In [109]:

# plan
# 1. get graph from PyTorch Geom
# 2. transform to nx (pyTorchGeometricDatasetToNx)
# 3. call createRandomGraphDataset(ba,'ba_1000_5','edge') 
#      or createRandomGraphDataset(ba,'ba_1000_5','node')


def processDatasetsSingle(dname, dataset):
    G = dataset.data
    print(G)
    g = pyTorchGeometricDatasetToNx(G,i)
    i+=1
    print(i)
    createRandomGraphDataset(g,'./precomputed/'+dname+'_'+str(i)+'_eb','edge', undirected=False) 
    createRandomGraphDataset(g,'./precomputed/'+dname+'_'+str(i)+'_nd','node') 

        
def processDatasets(dname, dataset):
    # set size of batch to total size of graph here
    loader = DataLoader(dataset, shuffle=False)
    i = 0
    for G in loader:
        print(G)
        g = pyTorchGeometricDatasetToNx(G,i)
        i+=1
        print(i)
        createRandomGraphDataset(g,'./precomputed/'+dname+'_'+str(i)+'_eb','edge', undirected=False) 
        createRandomGraphDataset(g,'./precomputed/'+dname+'_'+str(i)+'_nd','node') 


#KarateClub
print("\nKarateClub")
dname='KarateClub'
dataset = loadDataset(dname)
processDatasets(dname, dataset)

#ENZYMES FROM TUDataset
print("\nTUDataset EnZYMES")
dname='TUDataset'
name='ENZYMES'
dataset = loadDataset(dname,name)
processDatasets(dname+'_'+name, dataset)

#PROTEINS FROM TUDataset
print("\nTUDataset PROTEINS")
dname='TUDataset'
name='PROTEINS'
dataset = loadDataset(dname,name)
processDatasets(dname+'_'+name, dataset)

#QM7b
print("\QM7B QM7B")
dataset = loadDataset('QM7b')
processDatasets(dname,dataset)

# Planetoid Cora
print("\n Planetoid Cora")
dname='Planetoid'
name='Cora'
dataset = loadDataset(dname,name)
processDatasets(dname+'_'+name,dataset)



#---------------------------------------------------------------------


#MUTAG
#print("\n MUTAG")
#dname='Entities'
#name='MUTAG'
#dataset = loadDataset(dname,name)
#processDatasets(dname+'_'+name,dataset)



#PPI
#print("\PPI PPI")
#dname='PPI'
#dataset = loadDataset(dname)
#processDatasets(dname,dataset)



        
        



KarateClub
Batch(batch=[34], edge_index=[2, 156], x=[34, 34], y=[34])
1
1
Data(edge_index=[2, 78], x=[78, 1], y=[78])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 156], x=[34, 1], y=[34])
 directed graph:  False
isolated nodes:  False
self loops:  False

TUDataset EnZYMES
Extracting data/TUDataset/ENZYMES.zip
Processing...
Done!
Batch(batch=[37], edge_index=[2, 168], x=[37, 3], y=[1])
1
1
Data(edge_index=[2, 84], x=[84, 1], y=[84])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 168], x=[37, 1], y=[37])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[23], edge_index=[2, 102], x=[23, 3], y=[1])
2
1
Data(edge_index=[2, 51], x=[51, 1], y=[51])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 102], x=[23, 1], y=[23])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[25], edge_index=[2, 92], x=[25, 3], y=[1])
3
1
Dat

1
Data(edge_index=[2, 86], x=[24, 1], y=[24])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[24], edge_index=[2, 100], x=[24, 3], y=[1])
40
1
Data(edge_index=[2, 50], x=[50, 1], y=[50])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 100], x=[24, 1], y=[24])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[47], edge_index=[2, 190], x=[47, 3], y=[1])
41
1
Data(edge_index=[2, 95], x=[95, 1], y=[95])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 190], x=[47, 1], y=[47])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[45], edge_index=[2, 194], x=[45, 3], y=[1])
42
1
Data(edge_index=[2, 97], x=[97, 1], y=[97])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 194], x=[45, 1], y=[45])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[45], edge_index=[2, 190], 

 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 176], x=[38, 1], y=[38])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[40], edge_index=[2, 140], x=[40, 3], y=[1])
72
1
Data(edge_index=[2, 70], x=[70, 1], y=[70])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 140], x=[40, 1], y=[40])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[40], edge_index=[2, 174], x=[40, 3], y=[1])
73
1
Data(edge_index=[2, 87], x=[87, 1], y=[87])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 174], x=[40, 1], y=[40])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[42], edge_index=[2, 188], x=[42, 3], y=[1])
74
1
Data(edge_index=[2, 94], x=[94, 1], y=[94])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 188], x=[42, 1], y=[42])
 directed graph:  False
isolated nodes: 

1
Data(edge_index=[2, 29], x=[29, 1], y=[29])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 58], x=[16, 1], y=[16])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[26], edge_index=[2, 98], x=[26, 3], y=[1])
111
1
Data(edge_index=[2, 49], x=[49, 1], y=[49])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 98], x=[26, 1], y=[26])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[51], edge_index=[2, 190], x=[51, 3], y=[1])
112
1
Data(edge_index=[2, 95], x=[95, 1], y=[95])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 190], x=[51, 1], y=[51])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[52], edge_index=[2, 196], x=[52, 3], y=[1])
113
1
Data(edge_index=[2, 98], x=[98, 1], y=[98])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 196], x=[52, 1], y=

1
Data(edge_index=[2, 82], x=[29, 1], y=[29])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[22], edge_index=[2, 80], x=[22, 3], y=[1])
151
1
Data(edge_index=[2, 40], x=[40, 1], y=[40])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 80], x=[22, 1], y=[22])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[11], edge_index=[2, 44], x=[11, 3], y=[1])
152
1
Data(edge_index=[2, 22], x=[22, 1], y=[22])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 44], x=[11, 1], y=[11])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[8], edge_index=[2, 32], x=[8, 3], y=[1])
153
1
Data(edge_index=[2, 16], x=[16, 1], y=[16])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 32], x=[8, 1], y=[8])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[13], edge_index=[2, 44], x=[13, 3

...

1
Data(edge_index=[2, 160], x=[60, 1], y=[60])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[39], edge_index=[2, 144], x=[39, 3], y=[1])
485
1
Data(edge_index=[2, 72], x=[72, 1], y=[72])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 144], x=[39, 1], y=[39])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[26], edge_index=[2, 102], x=[26, 3], y=[1])
486
1
Data(edge_index=[2, 51], x=[51, 1], y=[51])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 102], x=[26, 1], y=[26])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[15], edge_index=[2, 54], x=[15, 3], y=[1])
487
1
Data(edge_index=[2, 27], x=[27, 1], y=[27])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 54], x=[15, 1], y=[15])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[25], edge_index=[2, 84],

 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 164], x=[37, 1], y=[37])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[48], edge_index=[2, 222], x=[48, 3], y=[1])
523
1
Data(edge_index=[2, 111], x=[111, 1], y=[111])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 222], x=[48, 1], y=[48])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[41], edge_index=[2, 148], x=[41, 3], y=[1])
524
1
Data(edge_index=[2, 74], x=[74, 1], y=[74])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 148], x=[41, 1], y=[41])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[29], edge_index=[2, 132], x=[29, 3], y=[1])
525
1
Data(edge_index=[2, 66], x=[66, 1], y=[66])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 132], x=[29, 1], y=[29])
 directed graph:  False
isolated n

1
Data(edge_index=[2, 48], x=[48, 1], y=[48])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 96], x=[26, 1], y=[26])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[52], edge_index=[2, 198], x=[52, 3], y=[1])
557
1
Data(edge_index=[2, 99], x=[99, 1], y=[99])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 198], x=[52, 1], y=[52])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[50], edge_index=[2, 186], x=[50, 3], y=[1])
558
1
Data(edge_index=[2, 93], x=[93, 1], y=[93])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 186], x=[50, 1], y=[50])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[38], edge_index=[2, 134], x=[38, 3], y=[1])
559
1
Data(edge_index=[2, 67], x=[67, 1], y=[67])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 134], x=[38, 1], 

 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[44], edge_index=[2, 180], x=[44, 3], y=[1])
595
1
Data(edge_index=[2, 90], x=[90, 1], y=[90])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 180], x=[44, 1], y=[44])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[45], edge_index=[2, 182], x=[45, 3], y=[1])
596
1
Data(edge_index=[2, 91], x=[91, 1], y=[91])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 182], x=[45, 1], y=[45])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[52], edge_index=[2, 232], x=[52, 3], y=[1])
597
1
Data(edge_index=[2, 116], x=[116, 1], y=[116])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 232], x=[52, 1], y=[52])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[55], edge_index=[2, 200], x=[55, 3], y=[1])
598
1
Data(edge_index=

TypeError: __init__() missing 1 required positional argument: 'root'

In [110]:

#PROTEINS FROM TUDataset
print("\nTUDataset PROTEINS")
dname='TUDataset'
name='PROTEINS'
dataset = loadDataset(dname,name)
processDatasets(dname+'_'+name, dataset)

#QM7b
print("\QM7B QM7B")
dataset = loadDataset('QM7b')
processDatasets(dname,dataset)

# Planetoid Cora
print("\n Planetoid Cora")
dname='Planetoid'
name='Cora'
dataset = loadDataset(dname,name)
processDatasets(dname+'_'+name,dataset)






TUDataset PROTEINS
Extracting data/TUDataset/PROTEINS.zip
Processing...
Done!
Batch(batch=[42], edge_index=[2, 162], x=[42, 3], y=[1])
1
1
Data(edge_index=[2, 81], x=[81, 1], y=[81])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 162], x=[42, 1], y=[42])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[27], edge_index=[2, 92], x=[27, 3], y=[1])
2
1
Data(edge_index=[2, 46], x=[46, 1], y=[46])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 92], x=[27, 1], y=[27])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[10], edge_index=[2, 34], x=[10, 3], y=[1])
3
1
Data(edge_index=[2, 17], x=[17, 1], y=[17])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 34], x=[10, 1], y=[10])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[24], edge_index=[2, 92], x=[24, 3], y=[1])
4
1
Data(edge_index=[2,

1
Data(edge_index=[2, 74], x=[74, 1], y=[74])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 148], x=[33, 1], y=[33])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[89], edge_index=[2, 286], x=[89, 3], y=[1])
40
1
Data(edge_index=[2, 143], x=[143, 1], y=[143])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 286], x=[89, 1], y=[89])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[50], edge_index=[2, 174], x=[50, 3], y=[1])
41
1
Data(edge_index=[2, 87], x=[87, 1], y=[87])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 174], x=[50, 1], y=[50])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[15], edge_index=[2, 50], x=[15, 3], y=[1])
42
1
Data(edge_index=[2, 25], x=[25, 1], y=[25])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 50], x=[15, 1], y

1
Data(edge_index=[2, 150], x=[150, 1], y=[150])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 300], x=[72, 1], y=[72])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[76], edge_index=[2, 240], x=[76, 3], y=[1])
75
1
Data(edge_index=[2, 120], x=[120, 1], y=[120])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 240], x=[76, 1], y=[76])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[6], edge_index=[2, 22], x=[6, 3], y=[1])
76
1
Data(edge_index=[2, 11], x=[11, 1], y=[11])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 22], x=[6, 1], y=[6])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[620], edge_index=[2, 2098], x=[620, 3], y=[1])
77
1
Data(edge_index=[2, 1049], x=[1049, 1], y=[1049])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 2098], x=[

isolated nodes:  False
self loops:  False
Batch(batch=[11], edge_index=[2, 42], x=[11, 3], y=[1])
106
1
Data(edge_index=[2, 21], x=[21, 1], y=[21])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 42], x=[11, 1], y=[11])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[244], edge_index=[2, 782], x=[244, 3], y=[1])
107
1
Data(edge_index=[2, 391], x=[391, 1], y=[391])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 782], x=[244, 1], y=[244])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[80], edge_index=[2, 328], x=[80, 3], y=[1])
108
1
Data(edge_index=[2, 164], x=[164, 1], y=[164])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 328], x=[80, 1], y=[80])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[22], edge_index=[2, 100], x=[22, 3], y=[1])
109
1
Data(edge_index=[2, 50], x=[50, 1],

1
Data(edge_index=[2, 216], x=[216, 1], y=[216])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 432], x=[122, 1], y=[122])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[28], edge_index=[2, 98], x=[28, 3], y=[1])
141
1
Data(edge_index=[2, 49], x=[49, 1], y=[49])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 98], x=[28, 1], y=[28])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[51], edge_index=[2, 190], x=[51, 3], y=[1])
142
1
Data(edge_index=[2, 95], x=[95, 1], y=[95])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 190], x=[51, 1], y=[51])
 directed graph:  False
isolated nodes:  False
self loops:  False
Batch(batch=[99], edge_index=[2, 312], x=[99, 3], y=[1])
143
1
Data(edge_index=[2, 156], x=[156, 1], y=[156])
 directed graph:  True
isolated nodes:  True
self loops:  False
1
Data(edge_index=[2, 312], x=[9

...

 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[13], edge_attr=[169], edge_index=[2, 169], y=[1, 14])
7004
1
Data(edge_index=[2, 90], x=[90, 1], y=[90])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 180], x=[13, 1], y=[13])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[12], edge_attr=[144], edge_index=[2, 144], y=[1, 14])
7005
1
Data(edge_index=[2, 77], x=[77, 1], y=[77])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 154], x=[12, 1], y=[12])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[10], edge_attr=[100], edge_index=[2, 100], y=[1, 14])
7006
1
Data(edge_index=[2, 54], x=[54, 1], y=[54])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 108], x=[10, 1], y=[10])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[14], edge_attr=[196], edge_index=[2, 196], 

 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 270], x=[16, 1], y=[16])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[13], edge_attr=[169], edge_index=[2, 169], y=[1, 14])
7051
1
Data(edge_index=[2, 90], x=[90, 1], y=[90])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 180], x=[13, 1], y=[13])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[15], edge_attr=[225], edge_index=[2, 225], y=[1, 14])
7052
1
Data(edge_index=[2, 119], x=[119, 1], y=[119])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 238], x=[15, 1], y=[15])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[15], edge_attr=[225], edge_index=[2, 225], y=[1, 14])
7053
1
Data(edge_index=[2, 119], x=[119, 1], y=[119])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 238], x=[15, 1], y=[15])
 dire

self loops:  True
Batch(batch=[17], edge_attr=[289], edge_index=[2, 289], y=[1, 14])
7094
1
Data(edge_index=[2, 152], x=[152, 1], y=[152])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 304], x=[17, 1], y=[17])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[15], edge_attr=[225], edge_index=[2, 225], y=[1, 14])
7095
1
Data(edge_index=[2, 119], x=[119, 1], y=[119])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 238], x=[15, 1], y=[15])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[14], edge_attr=[196], edge_index=[2, 196], y=[1, 14])
7096
1
Data(edge_index=[2, 104], x=[104, 1], y=[104])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 208], x=[14, 1], y=[14])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[12], edge_attr=[144], edge_index=[2, 144], y=[1, 14])
7097
1
Data(edge_index=[2, 

1
Data(edge_index=[2, 154], x=[12, 1], y=[12])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[11], edge_attr=[121], edge_index=[2, 121], y=[1, 14])
7137
1
Data(edge_index=[2, 65], x=[65, 1], y=[65])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 130], x=[11, 1], y=[11])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[11], edge_attr=[121], edge_index=[2, 121], y=[1, 14])
7138
1
Data(edge_index=[2, 65], x=[65, 1], y=[65])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 130], x=[11, 1], y=[11])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[12], edge_attr=[144], edge_index=[2, 144], y=[1, 14])
7139
1
Data(edge_index=[2, 77], x=[77, 1], y=[77])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 154], x=[12, 1], y=[12])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(bat

7190
1
Data(edge_index=[2, 90], x=[90, 1], y=[90])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 180], x=[13, 1], y=[13])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[13], edge_attr=[169], edge_index=[2, 169], y=[1, 14])
7191
1
Data(edge_index=[2, 90], x=[90, 1], y=[90])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 180], x=[13, 1], y=[13])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[15], edge_attr=[225], edge_index=[2, 225], y=[1, 14])
7192
1
Data(edge_index=[2, 119], x=[119, 1], y=[119])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Data(edge_index=[2, 238], x=[15, 1], y=[15])
 directed graph:  False
isolated nodes:  False
self loops:  True
Batch(batch=[10], edge_attr=[100], edge_index=[2, 100], y=[1, 14])
7193
1
Data(edge_index=[2, 54], x=[54, 1], y=[54])
 directed graph:  True
isolated nodes:  True
self loops:  True
1
Da